In [ ]:
import pandas as pd
import numpy as np
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction import DictVectorizer
from sklearn .svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from nltk.tokenize import word_tokenize


import matplotlib.pyplot as plt
import seaborn as sn

#LGBM:
import lightgbm as lgbm
import pandas as pd
import sklearn
import sklearn.metrics
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.preprocessing import StandardScaler, FunctionTransformer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Load Data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!ls

sample_data


Having all features that could be informative.

In [ ]:
# Training
df_train_preprocess = pd.read_csv(
    'trainset_final (1).csv', sep=';')#,header=True)

# Dev
df_dev_preprocess = pd.read_csv(
    'devset_final.csv', sep=';')#,header=True)

# Test
df_test_preprocess = pd.read_csv(
    'testset_final.csv', sep=';')

df_train = df_train_preprocess
df_dev = df_dev_preprocess
df_test = df_test_preprocess

df_train['POS'] = df_train[['POS_CD', 'POS_DT', 'POS_EX', 'POS_IN', 'POS_JJ', 'POS_JJR', 'POS_JJS', 'POS_MD', 'POS_NNP', 'POS_NNPS', 'POS_NNS', 'POS_PDT', 'POS_PRP', 'POS_PRP$', 'POS_RB', 'POS_RBR', 'POS_RBS', 'POS_RP', 'POS_TO', 'POS_UH', 'POS_VB', 'POS_VBD', 'POS_VBG', 'POS_VBN', 'POS_VBP', 'POS_VBZ', 'POS_WDT', 'POS_WP', 'POS_WP$', 'POS_WRB']].idxmax(axis=1).astype(dtype="category")
df_train['NE'] = df_train[['Named_Entity_GPE', 'Named_Entity_LOCATION', 'Named_Entity_ORGANIZATION', 'Named_Entity_PERSON']].idxmax(axis=1).astype(dtype="category")
df_train = df_train.drop(['Named_Entity_GPE', 'Named_Entity_LOCATION', 'Named_Entity_ORGANIZATION', 'Named_Entity_PERSON', 'id', 'Sentence', 'Word_N', 'Token_N', 'Named_Entity_GSP', 'POS_FW', 'POS_NN', 'POS_CC', 'POS_LS', 'POS_SYM', 'POS_CD', 'POS_DT', 'POS_EX', 'POS_IN', 'POS_JJ', 'POS_JJR', 'POS_JJS', 'POS_MD', 'POS_NNP', 'POS_NNPS', 'POS_NNS', 'POS_PDT', 'POS_PRP', 'POS_PRP$', 'POS_RB', 'POS_RBR', 'POS_RBS', 'POS_RP', 'POS_TO', 'POS_UH', 'POS_VB', 'POS_VBD', 'POS_VBG', 'POS_VBN', 'POS_VBP', 'POS_VBZ', 'POS_WDT', 'POS_WP', 'POS_WP$', 'POS_WRB'], axis =1)

df_dev['POS'] = df_dev[['POS_CD', 'POS_DT', 'POS_EX', 'POS_IN', 'POS_JJ', 'POS_JJR', 'POS_JJS', 'POS_MD', 'POS_NNP', 'POS_NNPS', 'POS_NNS', 'POS_PDT', 'POS_PRP', 'POS_PRP$', 'POS_RB', 'POS_RBR', 'POS_RBS', 'POS_RP', 'POS_TO', 'POS_UH', 'POS_VB', 'POS_VBD', 'POS_VBG', 'POS_VBN', 'POS_VBP', 'POS_VBZ', 'POS_WDT', 'POS_WP', 'POS_WP$', 'POS_WRB']].idxmax(axis=1).astype(dtype="category")
df_dev = df_dev.drop(['id', 'Sentence', 'Word_N', 'Token_N', 'Named_Entity_GSP','POS_FW', 'POS_NN','POS_CD', 'POS_DT', 'POS_EX', 'POS_IN', 'POS_JJ', 'POS_JJR', 'POS_JJS', 'POS_MD', 'POS_NNP', 'POS_NNPS', 'POS_NNS', 'POS_PDT', 'POS_PRP', 'POS_PRP$', 'POS_RB', 'POS_RBR', 'POS_RBS', 'POS_RP', 'POS_TO', 'POS_UH', 'POS_VB', 'POS_VBD', 'POS_VBG', 'POS_VBN', 'POS_VBP', 'POS_VBZ', 'POS_WDT', 'POS_WP', 'POS_WP$', 'POS_WRB'], axis =1)

df_test['POS'] = df_test[['POS_CD', 'POS_DT', 'POS_EX', 'POS_IN', 'POS_JJ', 'POS_JJR', 'POS_JJS', 'POS_MD', 'POS_NNP', 'POS_NNPS', 'POS_NNS', 'POS_PDT', 'POS_PRP', 'POS_PRP$', 'POS_RB', 'POS_RBR', 'POS_RBS', 'POS_RP', 'POS_TO', 'POS_UH', 'POS_VB', 'POS_VBD', 'POS_VBG', 'POS_VBN', 'POS_VBP', 'POS_VBZ', 'POS_WDT', 'POS_WP', 'POS_WP$', 'POS_WRB']].idxmax(axis=1).astype(dtype="category")
df_test['NE'] = df_test[['Named_Entity_GPE', 'Named_Entity_LOCATION', 'Named_Entity_ORGANIZATION', 'Named_Entity_PERSON']].idxmax(axis=1).astype(dtype="category")
df_test = df_test.drop(['Named_Entity_GPE', 'Named_Entity_LOCATION', 'Named_Entity_ORGANIZATION', 'Named_Entity_PERSON','id', 'Sentence', 'Word_N', 'Token_N', 'POS_NN', 'POS_CD', 'POS_DT', 'POS_EX', 'POS_IN', 'POS_JJ', 'POS_JJR', 'POS_JJS', 'POS_MD', 'POS_NNP', 'POS_NNPS', 'POS_NNS', 'POS_PDT', 'POS_PRP', 'POS_PRP$', 'POS_RB', 'POS_RBR', 'POS_RBS', 'POS_RP', 'POS_TO', 'POS_UH', 'POS_VB', 'POS_VBD', 'POS_VBG', 'POS_VBN', 'POS_VBP', 'POS_VBZ', 'POS_WDT', 'POS_WP', 'POS_WP$', 'POS_WRB'], axis =1)

print(df_train.shape)
print(df_test.shape)
print(df_dev.shape)

FileNotFoundError: ignored

Remove embedding features

In [ ]:
df_train = df_train.drop(['Lemma_embedding_1', 'Lemma_embedding_2',
       'Lemma_embedding_3', 'Lemma_embedding_4', 'Lemma_embedding_5',
       'next_1_0', 'next_1_1', 'next_1_2', 'next_1_3', 'next_1_4', 'next_2_0',
       'next_2_1', 'next_2_2', 'next_2_3', 'next_2_4', 'next_3_0', 'next_3_1',
       'next_3_2', 'next_3_3', 'next_3_4', 'previous_1_0', 'previous_1_1',
       'previous_1_2', 'previous_1_3', 'previous_1_4', 'previous_2_0',
       'previous_2_1', 'previous_2_2', 'previous_2_3', 'previous_2_4',
       'previous_3_0', 'previous_3_1', 'previous_3_2', 'previous_3_3',
       'previous_3_4'], axis=1)

df_dev = df_dev.drop(['Lemma_embedding_1', 'Lemma_embedding_2',
       'Lemma_embedding_3', 'Lemma_embedding_4', 'Lemma_embedding_5',
       'next_1_0', 'next_1_1', 'next_1_2', 'next_1_3', 'next_1_4', 'next_2_0',
       'next_2_1', 'next_2_2', 'next_2_3', 'next_2_4', 'next_3_0', 'next_3_1',
       'next_3_2', 'next_3_3', 'next_3_4', 'previous_1_0', 'previous_1_1',
       'previous_1_2', 'previous_1_3', 'previous_1_4', 'previous_2_0',
       'previous_2_1', 'previous_2_2', 'previous_2_3', 'previous_2_4',
       'previous_3_0', 'previous_3_1', 'previous_3_2', 'previous_3_3',
       'previous_3_4'], axis=1)

df_test = df_test.drop(['Lemma_embedding_1', 'Lemma_embedding_2',
       'Lemma_embedding_3', 'Lemma_embedding_4', 'Lemma_embedding_5',
       'next_1_0', 'next_1_1', 'next_1_2', 'next_1_3', 'next_1_4', 'next_2_0',
       'next_2_1', 'next_2_2', 'next_2_3', 'next_2_4', 'next_3_0', 'next_3_1',
       'next_3_2', 'next_3_3', 'next_3_4', 'previous_1_0', 'previous_1_1',
       'previous_1_2', 'previous_1_3', 'previous_1_4', 'previous_2_0',
       'previous_2_1', 'previous_2_2', 'previous_2_3', 'previous_2_4',
       'previous_3_0', 'previous_3_1', 'previous_3_2', 'previous_3_3',
       'previous_3_4'], axis=1)

Drop min_token_span and has_correct_antonym

In [ ]:
df_train = df_train.drop(['Min_token_span', 'Has_correct_antonym'], axis=1)
df_dev = df_dev.drop(['Min_token_span', 'Has_correct_antonym'], axis=1)
df_test = df_test.drop(['Min_token_span', 'Has_correct_antonym'], axis=1)

# Training and Testing


In [ ]:
y_train = df_train['Label']
x_train = df_train.drop('Label', axis=1)
y_val = df_dev['Label']
x_val = df_dev.drop('Label', axis=1)
y_test = df_test['Label']
x_test = df_test.drop('Label', axis=1)

classifier = lgbm.LGBMClassifier(n_estimators=100)

classifier.fit(x_train, y_train)

predictions = classifier.predict(x_test)

clsf_report1 = pd.DataFrame(classification_report(y_true = df_test['Label'], y_pred = predictions, output_dict = True)).transpose()
confusion_matrix = pd.crosstab(df_test['Label'], predictions, rownames=['Actual'], colnames = ['Predicted'])
sn.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g')
plt.suptitle('LGB'
)
plt.savefig('LGB.pdf')
plt.show()

feature_importances = classifier.feature_importances_ # Get feature importances
importances = pd.Series(feature_importances, x_train.columns)
fig, ax = plt.subplots()
importances.plot.bar(ax=ax)
ax.set_title("Feature importances")
plt.savefig('LGBM_feature importance.pdf')
clsf_report1

# Training LGBM

Hyper-parameter tuning

In [ ]:
clf_params = {
    'classifier__num_leaves':         sp_randint(7, 2095),
    'classifier__reg_alpha':          [0, 1e-1, 1, 2, 5, 7, 10],
    'classifier__reg_lambda':         [0, 1e-1, 1, 5, 10, 20],
    'classifier__max_depth':          sp_randint(2, 50),
    'classifier__objective':          ['cross_entropy', 'softmax']
}

In [ ]:
clf = Pipeline(
    steps=[
            ('classifier',  lgbm.LGBMClassifier())
        ]
    )

rscv = RandomizedSearchCV(
    clf,
    param_distributions = clf_params,
    cv = 5, # 80% train 20% val
    verbose = 1,
    n_jobs = 2
)

rscv.fit(x_train, y_train)

In [ ]:
classifier_LGB = rscv.best_estimator_
print("Best parameters rscv LGB:", rscv.best_params_)

In [ ]:
predictions = classifier_LGB.predict(x_test)

clsf_report1 = pd.DataFrame(classification_report(y_true = df_test['Label'], y_pred = predictions, output_dict = True)).transpose()
confusion_matrix = pd.crosstab(df_test['Label'], predictions, rownames=['Actual'], colnames = ['Predicted'])
sn.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g')
plt.suptitle('LGB')
plt.show()

feature_importances = classifier.feature_importances_ # Get feature importances
importances = pd.Series(feature_importances, x_train.columns)
fig, ax = plt.subplots()
importances.plot.bar(ax=ax)
ax.set_title("Feature importances")

clsf_report1